In [1]:
import pandas as pd
import json
import os
from tqdm import tqdm
import arrow
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
os.listdir('../')

['lab2-data-scraping-MMarciniak103', 'lab2-participation-MMarciniak103']

In [3]:
def get_user_df(topic: str):
    data = []
    
    default_path = f'../lab2-data-scraping-MMarciniak103/{topic}_users'
    new_path = default_path+"_new.json"
    new_replies = default_path+"_new_replies.json"
    
    with open(new_path) as f:
        for line in f:
            data.append(json.loads(line)['raw_value'])
    
    with open(new_replies) as f:
        for line in f:
            data.append(json.loads(line)['raw_value'])
    
    df = pd.DataFrame(data)
    return df

def get_tweets_df(topic: str, user_df: pd.DataFrame):
    data = []
    default_path = f'../lab2-data-scraping-MMarciniak103/{topic}_tweets' 
    new_path = default_path + "_new.json"
    new_replies = default_path + "_new_replies.json"
    
    with open(new_path) as f:
        for line in f:
            data.append(json.loads(line)['raw_value'])
    
    with open(new_replies) as f:
        for line in f:
            data.append(json.loads(line)['raw_value'])
    
    df = pd.DataFrame(data)
    fmt = r"ddd[\s+]MMM[\s+]DD[\s+]HH:mm:ss[\s+]+SSSS[\s+]YYYY"
    df['created_at'] = df['created_at'].apply(lambda x: arrow.get(x, fmt).datetime)
    df['username'] = df['user_id_str'].apply(lambda x: user_df[user_df['id_str'] == x].iloc[0]['name'])
    return df

def get_topic_df(topic: str):
    users_df = get_user_df(topic)
    tweets_df = get_tweets_df(topic, users_df)
    return (tweets_df, users_df)

def get_top_users(topic: str, users_df: pd.DataFrame, tweets_df: pd.DataFrame, N=40):
    
    top_users = []
    with open(f'../lab2-data-scraping-MMarciniak103/{topic}_top_{N}_users.txt', 'w') as f:
        for user_id in tqdm(tweets_df['user_id_str'].value_counts()[:N].keys()):
            user_object = users_df[users_df['id_str'] == user_id].iloc[0]
            f.write(f"{user_object['id_str']},{user_object['name']},{user_object['screen_name']}\n")
            top_users.append([user_object['id_str'], user_object['name'], user_object['screen_name']])
            
    return top_users

In [4]:
koronawirus_tweets, koronawirus_users = get_topic_df('koronawirus')

In [5]:
ekstraklasa_tweets, ekstraklasa_users = get_topic_df('ekstraklasa')

In [6]:
import plotly.graph_objects as go
import plotly.express as px

In [7]:
def aggregate_by_month(tweets_df: pd.DataFrame):
    aggregated_by_month = tweets_df[tweets_df['created_at'].dt.year > 2019].groupby(tweets_df['created_at'].dt.to_period("M")).agg('count')
    return aggregated_by_month

def aggregate_by_month_and_user(tweets_df: pd.DataFrame):
    aggregated_by_month = tweets_df[tweets_df['created_at'].dt.year > 2019].groupby([tweets_df['created_at'].dt.to_period("M"),tweets_df['username']]).agg('count')
    return aggregated_by_month
    
    
def get_posts_and_comments(tweets_df: pd.DataFrame):
    posts = tweets_df[tweets_df['in_reply_to_user_id_str'].isnull()]
    comments = tweets_df[tweets_df['in_reply_to_user_id_str'].notnull()]    
    
    return (posts, comments)


def posts_distribution(tweets_df: pd.DataFrame, topic):
    posts, comments = get_posts_and_comments(tweets_df)
    
    posts_aggregated = posts.groupby(posts['username']).agg('count')
    posts_aggregated = posts_aggregated.sort_values(by=['id'])
    
    
    fig = px.histogram(x=posts_aggregated.index, y=posts_aggregated['id'])
    fig.update_layout(
        height=600,
        width=1000,
        title=f"Rozkład liczby postów wśród użytkowników - {topic}",
        hovermode="x unified",
        yaxis_title="# Postów"
    )
    
    fig.show()
    
def comments_distribution(tweets_df: pd.DataFrame, topic):
    posts, comments = get_posts_and_comments(tweets_df)
    
    comments_aggregated = comments.groupby(comments['username']).agg('count')
    comments_aggregated = comments_aggregated.sort_values(by=['id'])
    
    
    fig = px.histogram(x=comments_aggregated.index, y=comments_aggregated['id'])
    fig.update_layout(
        height=600,
        width=1000,
        title=f"Rozkład liczby komentarzy wśród użytkowników - {topic}",
        hovermode="x unified",
        yaxis_title="# Komentarzy"
    )
    
    fig.show()

def post_dist_by_topic(df1, df2, topic1, topic2):
    
    posts1, comments1 = get_posts_and_comments(df1)
    posts2, comments2 = get_posts_and_comments(df2)
    
    posts1['category'] = topic1
    posts2['category'] = topic2
    
    posts = pd.concat([posts1, posts2])
    
    posts_aggregated = posts.groupby(posts['category']).agg('count')
    posts_aggregated = posts_aggregated.sort_values(by=['id'])
    
    fig = go.Figure()
    fig = px.bar(x=posts_aggregated.index, y=posts_aggregated['id'], color=posts_aggregated.index)
    
    fig.update_layout(
        height=600,
        width=1000,
        title=f"Całkowita liczba postów",
        hovermode="x unified",
        yaxis_title="# Postów"
    )
    
    fig.show()

def comments_dist_by_topic(df1, df2, topic1, topic2):
    
    posts1, comments1 = get_posts_and_comments(df1)
    posts2, comments2 = get_posts_and_comments(df2)
    
    comments1['category'] = topic1
    comments2['category'] = topic2
    
    posts = pd.concat([comments1, comments2])
    
    posts_aggregated = posts.groupby(posts['category']).agg('count')
    posts_aggregated = posts_aggregated.sort_values(by=['id'])
    
    fig = go.Figure()
    fig = px.bar(x=posts_aggregated.index, y=posts_aggregated['id'], color=posts_aggregated.index)
    
    fig.update_layout(
        height=600,
        width=1000,
        title=f"Całkowita liczba komentarzy",
        hovermode="x unified",
        yaxis_title="# Komentarzy"
    )
    
    fig.show()

In [9]:
import warnings
warnings.filterwarnings("ignore")

### Rozkład liczby postów opublikowanych przez użytkowników w zadanych tematach

In [10]:
post_dist_by_topic(koronawirus_tweets, ekstraklasa_tweets, 'koronawirus', 'ekstraklasa')

In [11]:
posts_distribution(koronawirus_tweets, 'koronawirus')

In [12]:
posts_distribution(ekstraklasa_tweets, 'ekstraklasa')

### Rozkład liczby komentarzy opublikowanych przez użytkowników w zadanych tematach

In [13]:
comments_dist_by_topic(koronawirus_tweets, ekstraklasa_tweets, 'koronawirus', 'ekstraklasa')

In [14]:
comments_distribution(koronawirus_tweets, 'koronawirus')

In [15]:
comments_distribution(ekstraklasa_tweets, 'ekstraklasa')

### Procent wszystkich użytkowników piszących posty w obu tematach

In [16]:
koronawirus_posts, koronawirus_comments = get_posts_and_comments(koronawirus_tweets)
ekstraklasa_posts, ekstraklasa_comments = get_posts_and_comments(ekstraklasa_tweets)

In [17]:
def both_topic_posts_creators(data: dict):
    
    topic_users = {}
    
    for topic, topic_data in data.items():
        topic_users[topic] = set(topic_data['username'])
        
    values_iterator = iter(topic_users.values())

    first_topic_users = next(values_iterator)
    second_topic_users = next(values_iterator)
    
    topic_users['both'] = first_topic_users.intersection(second_topic_users)
        
    df = pd.DataFrame({
        'topic': list(topic_users.keys()),
        'users_count': [len(users) for users in topic_users.values()]
    })
    fig = px.pie(df, values='users_count', names='topic', 
                 labels={
                     'users_count': 'Liczba użytkowników piszących posty na dany temat',
                     'topic': 'Temat'
                 },
                 title='Procentowy rozkład użytkowników piszących posty w danych tematach')
    fig.show()
    
def both_topic_comments_creators(data: dict):
    
    topic_users = {}
    
    for topic, topic_data in data.items():
        topic_users[topic] = set(topic_data['username'])
        
    values_iterator = iter(topic_users.values())

    first_topic_users = next(values_iterator)
    second_topic_users = next(values_iterator)
    
    topic_users['both'] = first_topic_users.intersection(second_topic_users)
        
    df = pd.DataFrame({
        'topic': list(topic_users.keys()),
        'users_count': [len(users) for users in topic_users.values()]
    })
    fig = px.pie(df, values='users_count', names='topic', 
                 labels={
                     'users_count': 'Liczba użytkowników piszących komentarze na dany temat',
                     'topic': 'Temat'
                 },
                 title='Procentowy rozkład użytkowników piszących komentarze w danych tematach')
    fig.show()

In [18]:
both_topic_posts_creators({"ektraklasa": ekstraklasa_posts, "koronawirus": koronawirus_posts})

In [19]:
both_topic_comments_creators({"ektraklasa": ekstraklasa_comments, "koronawirus": koronawirus_comments})

### Rozkład łącznej aktywności (suma reakcji, komentarzy, udostępnień) dotyczącej postów w zadanych tematach

In [20]:
def aggregate_activity(data):
    tweets = data[data['in_reply_to_user_id_str'].isnull()]
    tweets_grouped = tweets.groupby('created_at').agg({'favorite_count':'sum','reply_count':'sum',
                                                       'retweet_count':'sum', 'quote_count': 'sum'})
    
    tweets_grouped['activity_sum'] = tweets_grouped.sum(axis=1)
    tweets_grouped = tweets_grouped.drop(['favorite_count', 'reply_count', 
                                          'retweet_count', 'quote_count'], axis=1).reset_index()
    
    return tweets_grouped

def tweets_topic_activity_distribution_plot(df1, df2, topic1, topic2):
   
    df1['category'] = topic1
    df2['category'] = topic2
    
    posts = pd.concat([df1, df2])  
    
    posts_grouped = posts.groupby('category').agg({'favorite_count':'sum','reply_count':'sum',
                                                       'retweet_count':'sum', 'quote_count': 'sum'})
    
    posts_grouped['activity_sum'] = posts_grouped.sum(axis=1)
    posts_grouped = posts_grouped.drop(['favorite_count', 'reply_count', 
                                          'retweet_count', 'quote_count'], axis=1)
    
    fig = go.Figure()
    fig = px.bar(x=posts_grouped.index, y=posts_grouped['activity_sum'], color=posts_grouped.index)
    
    fig.update_layout(
        height=600,
        width=1000,
        title=f"Całkowita liczba aktywności w tematach",
        hovermode="x unified",
        yaxis_title="Aktywność"
    )
    
    fig.show()

In [21]:
tweets_topic_activity_distribution_plot(ekstraklasa_tweets, koronawirus_tweets, 'ekstraklasa', 'koronawirus')

#### Podsumowanie

Oba tematy cieszą się podobną charakterystyką jeśli chodzi o całkowitą liczbę postów. Podobny jest również rozkład postów wsród udzielających się użytkowników - dla obu tematów przeważają użytkownicy o małej liczbie postów z skupieniem większości aktywności wokół kilku profili (czy tak można opisać większość tematów na twiterze?).

Jeżeli chodzi o liczbę komentarzy to zdecydowanie przoduje temat koronowirusa. Rozkład aktywności użytkowników jest podobny jak w przypadku postów, aczkolwiek można dostrzec że prawy koniec wykresu się bardziej rozlewa niż w poprzednim przykładzie (Więcej użytkowników komentuje niż postuje). Sama dysproporcja pomiędzy koronawirusem a ekstraklasą może być spowodowana procesem pobierania danych. Koronawirus był pierwszy w kolejce, a zbieranie było wykonywane jedno po drugim. Być może tweeter ograniczył jakoś dostęp do danych dla pobierającego użytkownika.

Mało osób udziela się w obu tematach naraz (a przynajmniej nie zostało to uchwycone w 'nie idealnym' procesie zbierania danych). Wynik analizy jest podobny zarówno dla postów jak i komentarzy.

Natomiast pod kątem aktywności można zauważyć, że zdecydowanie na prowadzenie wyszedł temat koronawirusa. Jest to zapewne związane z tym, że wzbudza on duże emocje, zarówno negatywne jak i pozytywne (a wiadomo, że takie tematy przyciągają coraz to wieksze zainteresowanie).

### Podpunkt b


In [25]:
from sentimentpl.models import SentimentPLModel
from ipywidgets import IntProgress
from scipy.stats import pearsonr, spearmanr, kendalltau
from tqdm.auto import tqdm
tqdm.pandas()
import numpy as np
import os

In [26]:
sentiment_model = SentimentPLModel(from_pretrained='latest')

In [40]:
def calc_sentiment(text):
    return sentiment_model(text).item()

def analyze_sentiment(tweets_df, topic):
    if os.path.exists(f'{topic}_sentiment.csv'):
        return pd.read_csv(f'{topic}_sentiment.csv').drop(columns=['Unnamed: 0'])
    
    tweets_df['sentiment'] = tweets_df['full_text'].progress_map(calc_sentiment)
    tweets_df.to_csv(f'{topic}_sentiment.csv')
    return tweets_df

In [41]:
koronawirus_tweets= analyze_sentiment(koronawirus_tweets, 'koronawirus')

In [44]:
ekstraklasa_tweets = analyze_sentiment(ekstraklasa_tweets, 'ekstraklasa')

In [45]:
def sentiment_activity(df1, df2, topic1, topic2):
    
    corr_matrix = np.ndarray((2,4))
    
    corr_matrix[0,0] = spearmanr(df1['sentiment'], df1['favorite_count']).correlation
    corr_matrix[0,1] = spearmanr(df1['sentiment'], df1['reply_count']).correlation
    corr_matrix[0,2] = spearmanr(df1['sentiment'], df1['retweet_count']).correlation
    corr_matrix[0,3] = spearmanr(df1['sentiment'], df1['quote_count']).correlation
    
    corr_matrix[1,0] = spearmanr(df2['sentiment'], df2['favorite_count']).correlation
    corr_matrix[1,1] = spearmanr(df2['sentiment'], df2['reply_count']).correlation
    corr_matrix[1,2] = spearmanr(df2['sentiment'], df2['retweet_count']).correlation
    corr_matrix[1,3] = spearmanr(df2['sentiment'], df2['quote_count']).correlation
    
    fig = px.imshow(corr_matrix,
                    labels=dict(x="Aktywność", y="Temat", color="Korelacja między sentymentem, a aktywnością"),
                    x = ['Liczba polubień', 'Liczba odpowiedzi', 'Liczba retweetów', 'Liczba cytowań'],
                    y = [topic1, topic2])
    fig.update_xaxes(side="top")
    
    fig.show()

In [46]:
sentiment_activity(ekstraklasa_tweets, koronawirus_tweets, "ektraklasa", "koronawirus")

W przypadku tematu ekstraklasy można zauważyć, że najwieksza korelacja występuje pomiędzy pozytywnym wydźwiękiem a liczbą retweetów. Być może jest to związane z informacjami o wygraniu danego meczu lub udanym transferze co zwykle skutkuje udostępnianiem takiej informacji dalej przez użtykowników. W pozostałych aktywnościach cięzko stwierdzić istnienie takiej zależności ze względu na mały współczynnik korelacji.

W przypadku tematu koronawirusa również ciężko wskazać taką zależność ze względu na niskie wartości współczynnika korelacji Spearmana.

### Podpunkt c

In [47]:
import spacy
import pyLDAvis
import pyLDAvis.gensim_models

from tqdm import tqdm
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel
pl_nlp = spacy.load("pl_core_news_sm", disable=["parser", "ner"])

/home/podkowa/Polibuda/cyfrowe/lab2-data-scraping-MMarciniak103/cyfroweenv/lib/python3.8/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning:

`scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.



In [48]:
def clean_text(df):
    cleaned = []
    
    for text in tqdm(df['full_text']):
        doc = pl_nlp(text)
        cleaned.append(
            [
                token.lemma_.lower()
                for token in doc
                if not (
                    token.is_stop
                    or token.is_punct
                    or token.like_email
                    or token.like_url
                    or token.like_num
                    or token.is_digit
                    or token.pos_ not in ["NOUN", "ADJ", "VERB", "ADV"]
                )
            ]
        )
        
    return cleaned

def filter_data(data):
    id2word = Dictionary(data)
    id2word.filter_extremes(no_below=20, no_above=0.5)
    corpus = [id2word.doc2bow(text) for text in data]
    
    return id2word, corpus

In [49]:
def find_best_num_clasters(cleaned, topic):
    id2word, corpus = filter_data(cleaned)
    
    topics_num = [2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 18, 20]
    
    df = pd.DataFrame({
        'num_topics': [],
        'perplexity': [],
        'coherence': []
    })
    
    
    for topic_num in tqdm(topics_num):
        lda_model = LdaMulticore(corpus=corpus, id2word=id2word, num_topics=topic_num)
        
        perplexity = lda_model.log_perplexity(corpus)
        coherence_lda = CoherenceModel(
            model=lda_model, texts=cleaned, dictionary=id2word, coherence="c_v"
        ).get_coherence()
        
        df = df.append({
            'num_topics': topic_num,
            'perplexity': perplexity,
            'coherence': coherence_lda},
        ignore_index=True)
        
    fig = px.scatter(df, x="perplexity", y="coherence", color="num_topics", hover_name="num_topics", render_mode="svg", 
                 labels={
                     'perplexity': 'Perplexity',
                     'coherence': 'Coherence',
                     'topic': 'Liczba tematów'
                 },
                 title=f'Wpływ liczby tematów na wartości metryk - {topic}')

    fig.show()


In [50]:
cleaned_koronawirus = clean_text(koronawirus_tweets)

100%|████████████████████████████████████| 17769/17769 [02:08<00:00, 138.41it/s]


In [51]:
cleaned_ekstraklasa = clean_text(ekstraklasa_tweets)

100%|████████████████████████████████████| 12303/12303 [01:28<00:00, 139.48it/s]


In [52]:
find_best_num_clasters(cleaned_koronawirus, 'koronawirus')

100%|███████████████████████████████████████████| 13/13 [02:17<00:00, 10.56s/it]


In [53]:
find_best_num_clasters(cleaned_ekstraklasa, 'ekstraklasa')

100%|███████████████████████████████████████████| 13/13 [01:23<00:00,  6.41s/it]


In [59]:
def run_lda_interactive(cleaned, num_topics):
    id2word, corpus = filter_data(cleaned)
    
    lda_model = LdaMulticore(corpus=corpus, id2word=id2word, num_topics=num_topics)
    pyLDAvis.enable_notebook()
    
    vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    return vis

### koronawirus

In [60]:
run_lda_interactive(cleaned_koronawirus, 20)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
15    -0.079314  0.001974       1        1  6.481790
3      0.028555  0.022416       2        1  6.442541
1     -0.091548  0.015743       3        1  6.103686
7     -0.018210 -0.003416       4        1  5.821093
18     0.005723 -0.007747       5        1  5.577497
17     0.029307  0.012826       6        1  5.479694
9      0.033146 -0.046186       7        1  5.443376
4      0.006076 -0.007901       8        1  5.053109
0     -0.003762 -0.019209       9        1  4.901295
6      0.052911 -0.007049      10        1  4.828774
12    -0.010848 -0.002146      11        1  4.743672
16    -0.013885  0.017481      12        1  4.722617
8     -0.000077  0.016472      13        1  4.684364
19     0.012921 -0.009468      14        1  4.602169
5      0.057146  0.054705      15        1  4.581517
14     0.023988 -0.032207      16        1  4.524887
11     0.007983  0.025225      17        1  4.155844
13     0.010771 -0.083173      18        1  3.985671
2     -0.067593 -0.003135      19        1  3.965632
10     0.016710  0.054795      20        1  3.900775, topic_info=             Term         Freq        Total Category  logprob  loglift
187     zakażenie  1767.000000  1767.000000  Default  30.0000  30.0000
18           nowy  1637.000000  1637.000000  Default  29.0000  29.0000
79    szczepionka  1233.000000  1233.000000  Default  28.0000  28.0000
50      przypadek  1384.000000  1384.000000  Default  27.0000  27.0000
1446     pomorski   274.000000   274.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
187     zakażenie    33.205265  1767.741217  Topic20  -5.2207  -0.7308
295       szpital    27.635256   761.719442  Topic20  -5.4043  -0.0725
67          dzień    25.742712   690.218639  Topic20  -5.4752  -0.0449
805      pandemia    27.467779  1439.090800  Topic20  -5.4104  -0.7148
185        raport    24.087548   418.262606  Topic20  -5.5417   0.3896

[1372 rows x 6 columns], token_table=      Topic      Freq Term
term                      
1495      1  0.068733   +0
1495      2  0.247440   +0
1495      3  0.261186   +0
1495      6  0.041240   +0
1495      8  0.068733   +0
...     ...       ...  ...
420      14  0.027872    🦠
420      17  0.027872    🦠
420      18  0.027872    🦠
420      19  0.111486    🦠
420      20  0.027872    🦠

[11437 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 4, 2, 8, 19, 18, 10, 5, 1, 7, 13, 17, 9, 20, 6, 15, 12, 14, 3, 11])

In [61]:
run_lda_interactive(cleaned_koronawirus, 9)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.078676  0.014739       1        1  14.476035
1     -0.044679 -0.010480       2        1  12.181739
2     -0.012636 -0.029838       3        1  12.036717
8     -0.050797 -0.020365       4        1  11.821185
4     -0.017056  0.042903       5        1  11.027178
5     -0.013571  0.043267       6        1  10.029441
3      0.034753 -0.006232       7        1  10.017309
6     -0.015795  0.001872       8        1   9.672025
7      0.041105 -0.035866       9        1   8.738370, topic_info=              Term         Freq        Total Category  logprob  loglift
187      zakażenie  1769.000000  1769.000000  Default  30.0000  30.0000
18            nowy  1657.000000  1657.000000  Default  29.0000  29.0000
50       przypadek  1417.000000  1417.000000  Default  28.0000  28.0000
108          osoba  1771.000000  1771.000000  Default  27.0000  27.0000
1446      pomorski   293.000000   293.000000  Default  26.0000  26.0000
...            ...          ...          ...      ...      ...      ...
165   @morawieckim    63.264096   701.192944   Topic9  -5.3826   0.0320
195           czas    64.763255   865.443168   Topic9  -5.3592  -0.1551
514       minister    57.314966   457.288509   Topic9  -5.4814   0.3607
57         zdrowie    59.257286   996.166428   Topic9  -5.4480  -0.3846
172         polski    54.227734   579.856621   Topic9  -5.5367   0.0679

[635 rows x 6 columns], token_table=      Topic      Freq Term
term                      
1495      1  0.206512   +0
1495      2  0.110140   +0
1495      5  0.082605   +0
1495      6  0.261582   +0
1495      7  0.316652   +0
...     ...       ...  ...
1638      4  0.026758    🤬
1638      5  0.080275    🤬
1638      6  0.695719    🤬
1638      7  0.026758    🤬
1638      9  0.013379    🤬

[3327 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 9, 5, 6, 4, 7, 8])

In [65]:
run_lda_interactive(cleaned_koronawirus, 4)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.029151 -0.038326       1        1  29.049225
3     -0.032148  0.037538       2        1  25.784639
2      0.023071 -0.003921       3        1  23.061252
0      0.038228  0.004709       4        1  22.104884, topic_info=             Term         Freq        Total Category  logprob  loglift
89              ️   899.000000   899.000000  Default  30.0000  30.0000
108         osoba  1834.000000  1834.000000  Default  29.0000  29.0000
805      pandemia  1465.000000  1465.000000  Default  28.0000  28.0000
172        polski   599.000000   599.000000  Default  27.0000  27.0000
1533   małopolski   204.000000   204.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
166    @mz_gov_pl   164.739970   845.712954   Topic4  -5.3536  -0.1264
113         walka   150.859375   825.245866   Topic4  -5.4417  -0.1900
307        liczba   153.780036  1012.385425   Topic4  -5.4225  -0.3752
79    szczepionka   137.752194  1148.006819   Topic4  -5.5325  -0.6110
805      pandemia   128.651330  1465.580735   Topic4  -5.6009  -0.9235

[312 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1495      1  0.133212       +0
1495      2  0.093248       +0
1495      3  0.013321       +0
1495      4  0.759308       +0
1676      1  0.868733  @jsw_sa
...     ...       ...      ...
1804      4  0.045902  🟤liczba
1638      1  0.110278        🤬
1638      2  0.206770        🤬
1638      3  0.013785        🤬
1638      4  0.661665        🤬

[865 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 3, 1])

### ekstraklasa

In [66]:
run_lda_interactive(cleaned_ekstraklasa, 2)

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.017399  0.0       1        1  51.413716
0     -0.017399  0.0       2        1  48.586284, topic_info=              Term         Freq        Total Category  logprob  loglift
945              x   279.000000   279.000000  Default  30.0000  30.0000
108        kolejka   765.000000   765.000000  Default  29.0000  29.0000
42   @_ekstraklasa   569.000000   569.000000  Default  28.0000  28.0000
21     ekstraklasa  1992.000000  1992.000000  Default  27.0000  27.0000
272         lechia   236.000000   236.000000  Default  26.0000  26.0000
..             ...          ...          ...      ...      ...      ...
104         trener   205.655523   520.012454   Topic2  -5.2437  -0.2058
157              🤔   160.481093   319.676501   Topic2  -5.4917   0.0327
56               👏   177.583531   434.615007   Topic2  -5.3904  -0.1732
12         ostatni   171.125839   393.650005   Topic2  -5.4275  -0.1112
69        zawodnik   167.384897   395.804952   Topic2  -5.4496  -0.1388

[172 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
42        1  0.266941    @_ekstraklasa
42        2  0.734088    @_ekstraklasa
994       1  0.177775       @lvlaarcin
994       2  0.834174       @lvlaarcin
106       1  0.777612  @mkscracoviassa
...     ...       ...              ...
242       2  0.729930                😍
157       1  0.497378                🤔
157       2  0.500506                🤔
539       1  0.204827                🤦
539       2  0.778342                🤦

[244 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

In [67]:
run_lda_interactive(cleaned_ekstraklasa, 4)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.025936 -0.005311       1        1  32.490731
2     -0.032202 -0.034296       2        1  27.649299
3     -0.006942  0.053037       3        1  21.514464
1      0.065079 -0.013430       4        1  18.345506, topic_info=            Term         Freq        Total Category  logprob  loglift
18             ⚽  1059.000000  1059.000000  Default  30.0000  30.0000
945            x   276.000000   276.000000  Default  29.0000  29.0000
626           el   281.000000   281.000000  Default  28.0000  28.0000
97         legia  1261.000000  1261.000000  Default  27.0000  27.0000
55             ️  1655.000000  1655.000000  Default  26.0000  26.0000
..           ...          ...          ...      ...      ...      ...
5       transfer    78.885732   360.001432   Topic4  -5.2279   0.1777
21   ekstraklasa   157.667164  1985.098216   Topic4  -4.5354  -0.8372
23       piłkarz    83.195992   614.616161   Topic4  -5.1747  -0.3040
12       ostatni    72.794815   395.136326   Topic4  -5.3083   0.0042
45      pierwszy    65.575708   577.320173   Topic4  -5.4127  -0.4794

[281 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
532       1  0.106642            1:0
532       2  0.045704            1:0
532       3  0.517977            1:0
532       4  0.319927            1:0
42        1  0.499379  @_ekstraklasa
...     ...       ...            ...
993       4  0.217157              🤪
853       1  0.214327              🤯
853       2  0.071442              🤯
853       3  0.214327              🤯
853       4  0.500095              🤯

[774 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 4, 2])

In [68]:
def get_popular_unpopular(df):
    df['activity'] = df[['favorite_count', 'reply_count', 'retweet_count', 'quote_count']].sum(axis=1)
    sorted_df = df.sort_values(by=['activity'])
    popular = sorted_df.head(100)
    unpopular = sorted_df.tail(100)
    
    return (popular, unpopular)

def filter_data_no_extrema(data):
    id2word = Dictionary(data)
    corpus = [id2word.doc2bow(text) for text in data]
    
    return id2word, corpus

def compare_per_popularity(df, num_topics):
    popular, unpopular = get_popular_unpopular(df)
    
    popular_clean = clean_text(popular)
    unpopular_clean = clean_text(unpopular)
    
    popular_id2word, popular_corpus = filter_data_no_extrema(popular_clean)
    unpopular_id2word, unpopular_corpus = filter_data_no_extrema(unpopular_clean)
    
    
    lda_model_popular = LdaMulticore(corpus=popular_corpus, id2word=popular_id2word, num_topics=num_topics)
    lda_model_unpopular = LdaMulticore(corpus=unpopular_corpus, id2word=unpopular_id2word, num_topics=num_topics)
    
    
    popular_vis = pyLDAvis.gensim_models.prepare(lda_model_popular, popular_corpus, popular_id2word)
    unpopular_vis = pyLDAvis.gensim_models.prepare(lda_model_unpopular, unpopular_corpus, unpopular_id2word)

    return (popular_vis, unpopular_vis)

### koronawirus

In [72]:
pyLDAvis.enable_notebook()
popular, unpopular = compare_per_popularity(koronawirus_tweets, 4)

100%|████████████████████████████████████████| 100/100 [00:00<00:00, 154.11it/s]


In [73]:
popular

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.016316  0.069955       1        1  35.000576
1     -0.083435 -0.026026       2        1  32.619172
3      0.056617 -0.061834       3        1  18.481605
2      0.010502  0.017905       4        1  13.898647, topic_info=             Term      Freq     Total Category  logprob  loglift
73      zakażenie  3.000000  3.000000  Default  30.0000  30.0000
364           móc  1.000000  1.000000  Default  29.0000  29.0000
131      epidemia  4.000000  4.000000  Default  28.0000  28.0000
649  przeciwciało  1.000000  1.000000  Default  27.0000  27.0000
633  ograniczenie  1.000000  1.000000  Default  26.0000  26.0000
..            ...       ...       ...      ...      ...      ...
197      pierwszy  0.559802  2.323259   Topic4  -5.7417   0.5502
636     skuteczny  0.559797  1.717095   Topic4  -5.7417   0.8526
310         późno  0.559597  1.703045   Topic4  -5.7421   0.8604
78        wykonać  0.559418  3.005109   Topic4  -5.7424   0.2922
76       fałszywy  0.559347  1.656826   Topic4  -5.7425   0.8875

[247 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
175       1  0.765287         @dietlypl
563       2  0.827847  @gazeta_wyborcza
564       2  0.827818     @hanna_lisowa
20        1  0.317172      @morawieckim
20        2  0.475757      @morawieckim
...     ...       ...               ...
67        1  0.239431             życie
67        2  0.478862             życie
67        3  0.239431             życie
16        4  0.613923                 ➡
187       1  0.765281                 🍱

[276 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 4, 3])

In [74]:
unpopular

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.047893 -0.070649       1        1  31.173781
1     -0.051634  0.054194       2        1  27.743993
3      0.080955 -0.014284       3        1  23.097509
2      0.018572  0.030740       4        1  17.984717, topic_info=            Term      Freq      Total Category  logprob  loglift
168            🟥  4.000000   4.000000  Default  30.0000  30.0000
109       deaths  3.000000   3.000000  Default  29.0000  29.0000
82       pacjent  4.000000   4.000000  Default  28.0000  28.0000
459  szczepienie  2.000000   2.000000  Default  27.0000  27.0000
260          get  5.000000   5.000000  Default  26.0000  26.0000
..           ...       ...        ...      ...      ...      ...
103    zakażenie  0.984852   7.278632   Topic4  -5.6188  -0.2846
139         will  0.848298   3.595372   Topic4  -5.7681   0.2715
27            in  1.029489  12.456805   Topic4  -5.5745  -0.7776
449       people  0.920973   6.438243   Topic4  -5.6859  -0.2289
123         that  0.843047   6.862922   Topic4  -5.7743  -0.3812

[269 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
17        4  0.594449        2nd
51        2  0.361571      about
51        3  0.361571      about
51        4  0.361571      about
128       1  0.344342  according
...     ...       ...        ...
278       1  0.980847          ❗
206       1  0.606153          🇧
206       2  0.303077          🇧
16        3  0.885906          😪
168       4  0.826580          🟥

[322 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 4, 3])

### ekstraklasa

In [75]:
pyLDAvis.enable_notebook()
popular, unpopular = compare_per_popularity(ekstraklasa_tweets, 4)

100%|████████████████████████████████████████| 100/100 [00:00<00:00, 192.71it/s]


In [76]:
popular

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.069227  0.036509       1        1  31.781873
3     -0.050927 -0.066324       2        1  25.401839
2      0.085359 -0.037332       3        1  23.170353
0      0.034795  0.067147       4        1  19.645934, topic_info=              Term       Freq      Total Category  logprob  loglift
61               ️   6.000000   6.000000  Default  30.0000  30.0000
15   @_ekstraklasa   4.000000   4.000000  Default  29.0000  29.0000
85     ekstraklasa  17.000000  17.000000  Default  28.0000  28.0000
67               ⚽   3.000000   3.000000  Default  27.0000  27.0000
60               ✅   4.000000   4.000000  Default  26.0000  26.0000
..             ...        ...        ...      ...      ...      ...
142              🏻   0.676770   1.664830   Topic4  -5.5994   0.7272
516         nedpol   0.676360   2.802594   Topic4  -5.6000   0.2057
43           awans   0.675756   2.384222   Topic4  -5.6009   0.3665
46           piłka   0.675750   2.360828   Topic4  -5.6009   0.3764
136            gra   0.675678   2.362819   Topic4  -5.6010   0.3754

[246 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
590       4  0.601017           -&gt
480       4  0.600961             1⃣
586       3  0.600640            2:2
586       4  0.600640            2:2
15        2  0.201608  @_ekstraklasa
...     ...       ...            ...
66        1  0.293115              🔥
66        3  0.586231              🔥
123       1  0.420000              🙄
123       2  0.420000              🙄
123       3  0.210000              🙄

[258 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 3, 1])

In [77]:
unpopular

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.126456 -0.032238       1        1  29.549835
1     -0.007028  0.004731       2        1  24.439223
0      0.102580 -0.068274       3        1  23.039647
3      0.030904  0.095781       4        1  22.971296, topic_info=         Term      Freq     Total Category  logprob  loglift
9           🔥  8.000000  8.000000  Default  30.0000  30.0000
233  sportowy  4.000000  4.000000  Default  29.0000  29.0000
506       his  6.000000  6.000000  Default  28.0000  28.0000
72          😂  9.000000  9.000000  Default  27.0000  27.0000
78          🏻  3.000000  3.000000  Default  26.0000  26.0000
..        ...       ...       ...      ...      ...      ...
200         ⚪  0.742688  3.657055   Topic4  -5.5468  -0.1232
190     dzień  0.740771  3.012343   Topic4  -5.5494   0.0681
181   procent  0.739957  1.778655   Topic4  -5.5505   0.5939
202         🔴  0.739637  2.401099   Topic4  -5.5509   0.2934
25      sezon  0.739342  2.460494   Topic4  -5.5513   0.2686

[212 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
203       2  0.561770    @_ekstraklasa
203       4  0.561770    @_ekstraklasa
303       2  0.843299  @beavertownbeer
358       1  0.530604      @podolski10
358       4  0.530604      @podolski10
...     ...       ...              ...
17        4  0.473329                🔵
72        1  0.401935                😂
72        2  0.200967                😂
72        4  0.401935                😂
331       3  0.549560                😆

[212 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1, 4])

/home/podkowa/Polibuda/cyfrowe/lab2-data-scraping-MMarciniak103/cyfroweenv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/podkowa/Polibuda/cyfrowe/lab2-data-scraping-MMarciniak103/cyfroweenv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/podkowa/Polibuda/cyfrowe/lab2-data-scraping-MMarciniak103/cyfroweenv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/podkowa/Polibuda/cyfrowe/lab2-data-scraping-MMarciniak103/cyfroweenv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module

###  Podsumowanie

W ramach zadanych tematów można zauważyć, że więcej osób udziela się w sposób komentowania niż dodawania nowych postów. Tworzeniem postów zajmują się zazwyczaj instytucje czy 'większe profile' (w przypadku koronawirusa) lub dziennikarze/media (ekstraklasa). Natomiast można znaleźć więcej profili prywatnych wśród komentujących. Zdecydowanie większą aktywnością cieszy się temat koronawirusa.

W kwestii wpływu wydźwięku postu na aktywność użytkowników nie zanotowano znacznej korelacji. Jest to o tyle zadziwiające, że można było spodziewać się jej istnienia. Zazwyczaj posty wzbudzające dużo emocji (zwłaszcza negatywnych) pobudzają dużą dyskusję a wręcz kłótnię w komentarzach. 

W przypadku tematu koronawirusa można zauważyć, że wykorzystywanie konkretnych słów wpływa na popularność postów (Są to m.in. nowe przypadki, szczepienia, rząd, pandemia, zgon). W przypadku tematu ekstraklasy taki trend nie jest już widoczny (tj. rozkłady słów wśród popularnych i niepopularnych postów są zbliżone).

W przypadku prowadzenia tych analiz należy mieć na uwadze fakt, że sam proces pobierania danych nie był idealny i obarczony pewnymi wadami, które mogły wpłynąć na zakrzywienie rzeczywistości poprzez pobranie części tweetów zamiast próbki reprezentacyjnej ogólny charakter danego tematu. W takim wypadku nieważne jakich narzędzi użyjemy do przeprowadzenia badań, ponieważ działamy w myśl zasady 'garbage in garbage out'.

W kwestii aspektów, które nie zostały ujęte w badaniu a mogłyby mieć wpływ na partycypację użtkownikow można wskazać wykrywanie profili, które są obserwowane przez wielu użytkowników i udzielają się w danych tematach.